In [ ]:
from google.colab import files
files.upload()

Saving MACDONALDS_FINAL_BALANCED_DATA.xlsx to MACDONALDS_FINAL_BALANCED_DATA.xlsx


{'MACDONALDS_FINAL_BALANCED_DATA.xlsx': b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x00?\x00a]I:O\x01\x00\x00\x8f\x04\x00\x00\x13\x00\x00\x00[Content_Types].xml\xad\x94\xcbn\xc20\x10E\xf7\xfd\x8a\xc8\xdb*1tQU\x15\x81E\x1f\xcb\x16\xa9\xf4\x03\\{B,\x1c\xdb\xf2\x0c\x14\xfe\xbe\x93\xf0P[Q\xa0\x82M\xacd\xee\xdcs\xc7\x8e<\x18-\x1b\x97- \xa1\r\xbe\x14\xfd\xa2\'2\xf0:\x18\xeb\xa7\xa5x\x9f<\xe7w"CR\xde(\x17<\x94b\x05(F\xc3\xab\xc1d\x15\x013n\xf6X\x8a\x9a(\xdeK\x89\xba\x86Fa\x11"x\xaeT!5\x8a\xf85MeTz\xa6\xa6 oz\xbd[\xa9\x83\'\xf0\x94S\xeb!\x86\x83G\xa8\xd4\xdcQ\xf6\xb4\xe4\xcf\xeb \t\x1c\x8a\xeca-lY\xa5P1:\xab\x15q].\xbc\xf9E\xc97\x84\x82;;\r\xd66\xe25\x0b\x84\xdcKh+\x7f\x036}\xaf\xbc3\xc9\x1a\xc8\xc6*\xd1\x8bjX%M\xd0\xe3\x14"J\xd6\x17\x87]\xf6\xc4\x0cUe5\xb0\xc7\xbc\xe1\x96\x02\xda@\x06L\x1e\xd9\x12\x12Y\xd8e>\xc8\xd6!\xc1\xff\xe1\xdb=j\xbbO$.\x9dDZ9\xc0\xb3G\xc5\x98@\x19\xac\x01\xa8q\xc5\xda\xf4\x08\x99\xf8\x7f\x82\xf5\xb3\x7f6\xbf\xb39\x02\xfc\x0ci\xf6\x11\xc2\xec\xd2\xc3\xb6k\xd1(\xebO\xe0wb\x9

In [ ]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
df = pd.read_excel('MACDONALDS_FINAL_BALANCED_DATA.xlsx')

In [ ]:
df.head(5)

,review,label
0,Making customers wait for more than half and h...,negative
1,I visited the McDonald’s Jogeshwari outlet yes...,negative
2,Pathetic system..ordered on mcdelievry app.. e...,negative
3,Service was very late… they were doing Timepas...,negative
4,Worst McDonald's takes alot to prepare the ord...,negative


In [ ]:
print(df.shape)
print(df.dtypes)

(486, 2)
review    object
label     object
dtype: object


In [ ]:
df['label'].value_counts()

,count
label,
negative,243
positive,243


In [ ]:
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
negations = {'not', 'no', 'nor', 'never', "n't"}
stop_words = stop_words - negations

def preprocess_review(review):
    # convert to lowercase
    review = review.lower()

    # remove punctuation and numbers
    #This removes everything from the review except lowercase letters and spaces.
    review = re.sub(r'[^a-z\s]', '', review)

    # split into words
    words = review.split()

    # remove stopwords and apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

In [ ]:
df['clean_review'] = df['review'].astype(str).apply(preprocess_review)

In [ ]:
w2v_model = Word2Vec(
    sentences=sentences,   # uses tokenized reviews to learn word relationships
    vector_size=100,       # sets the size of each word embedding vector
    window=5,              # considers 5 surrounding words as context
    min_count=2,           # ignores words that appear less than 2 times
    workers=4              # uses 4 CPU cores to speed up training
)

In [ ]:
def sentence_vector(sentence, model):
    # split the sentence into individual words
    words = sentence.split()

    # get Word2Vec vectors only for words known to the model
    vectors = [model.wv[word] for word in words if word in model.wv]

    # if no word has a vector, return a zero vector
    if len(vectors) == 0:
        return np.zeros(model.vector_size)

    # take the average of all word vectors to represent the sentence
    return np.mean(vectors, axis=0)

This function turns a sentence into numbers by averaging the Word2Vec vectors of its words.

In [ ]:
x = np.array([
    sentence_vector(text, w2v_model)
    for text in df['review']
])

y = df['label']

convert each review into a Word2Vec sentence vector

target labels for sentiment classification

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,
                                               random_state=42,stratify=y)

stratify=y Ensures train and test sets keep the same positive/negative class ratio as the original dataset.

In [ ]:
logistic=LogisticRegression(max_iter=1000)
logistic.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

max_iter=1000 Sets enough iterations for Logistic Regression to fully converge and avoid early stopping errors.

In [ ]:
acc_logistic=accuracy_score(y_test,y_pred)*100
print('Accuracy:',acc_logistic)

Accuracy: 67.3469387755102


In [ ]:
confusion=confusion_matrix(y_test,y_pred)
print('Confusion Matrix:',confusion)

Confusion Matrix: [[31 18]
 [14 35]]


In [ ]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(logistic,x,y,cv=5)
print(scores)
print('mean accuracy:',scores.mean())

[0.71428571 0.49484536 0.59793814 0.69072165 0.63917526]
mean accuracy: 0.6273932253313697


In [ ]:
def predict_review(review_text, w2v_model, logistic_model):
    # preprocess input review
    clean_text = preprocess_review(review_text)

    # convert review to Word2Vec vector
    review_vector = sentence_vector(clean_text, w2v_model)

    # reshape for prediction
    review_vector = [review_vector]

    # predict label and confidence
    prediction = logistic_model.predict(review_vector)[0]
    probability = logistic_model.predict_proba(review_vector).max()

    return prediction, probability

In [ ]:
confusing_reviews = [
    "The product looks good but the quality is not great.",
    "It is okay, not bad but not very impressive either.",
    "I didn’t expect much, but it’s not terrible.",
    "Great job, it stopped working on the first day.",
    "If the price was lower, this would have been a good product.",
    "Customer support was helpful but the product itself is disappointing.",
    "It works, nothing special.",
    "Initially it seemed fine, later it turned out to be a bad decision."
]

In [ ]:
for review in confusing_reviews:
  label,prob=predict_review(review, w2v_model,logistic)
  print('\nReview:',review)
  print('prediction:',label)
  print('confidence:',round(prob*100,2),'%')


Review: The product looks good but the quality is not great.
prediction: positive
confidence: 50.46 %

Review: It is okay, not bad but not very impressive either.
prediction: positive
confidence: 50.33 %

Review: I didn’t expect much, but it’s not terrible.
prediction: positive
confidence: 50.02 %

Review: Great job, it stopped working on the first day.
prediction: negative
confidence: 50.5 %

Review: If the price was lower, this would have been a good product.
prediction: negative
confidence: 50.22 %

Review: Customer support was helpful but the product itself is disappointing.
prediction: negative
confidence: 50.14 %

Review: It works, nothing special.
prediction: negative
confidence: 50.72 %

Review: Initially it seemed fine, later it turned out to be a bad decision.
prediction: negative
confidence: 50.92 %


The model performs very well on clear sentiments but struggles on mixed or neutral reviews, which is a known limitation of bag-of-words based approaches.